In [1]:
import pyspark
print(pyspark.__version__)

3.5.3


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("NYCTaxiAnalysis").getOrCreate()
df_taxi = spark.read.option("header", "true").csv("input/Sample NYC Data.csv")

In [3]:
df_taxi.printSchema()
df_taxi.show(5)

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)

+--------------------+--------------------+---------+---------+------------------+---------------+----------------+---------------+----------------+---------------+-----------------+----------------+
|           medallion|        hack_license|vendor_id|rate_code|store_and_fwd_flag|pickup_datetime|dropoff_datetime|passenger_count|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|
+--------------------+--------------------+

In [4]:
# Create duration column
from pyspark.sql.functions import unix_timestamp, col

df_taxi = df_taxi.withColumn(
    "pickup_ts",
    unix_timestamp(col("pickup_datetime"), "dd-MM-yy HH:mm")
)

df_taxi = df_taxi.withColumn(
    "dropoff_ts",
    unix_timestamp(col("dropoff_datetime"), "dd-MM-yy HH:mm")
)

df_taxi = df_taxi.withColumn(
    "duration_sec",
    col("dropoff_ts") - col("pickup_ts")
)

In [5]:
df_taxi.select("duration_sec").summary("count", "min", "max", "mean", "stddev").show()

+-------+------------------+
|summary|      duration_sec|
+-------+------------------+
|  count|             99999|
|    min|                 0|
|    max|              9180|
|   mean| 650.6807068070681|
| stddev|469.78651920883175|
+-------+------------------+



In [6]:
# Keep rides with duration at least 0 second duration and at most 2.5h duration
df_taxi = df_taxi.filter((col("duration_sec") >= 0) & (col("duration_sec") <= 2.5*60*60))

In [7]:
df_taxi.select("duration_sec").summary("count", "min", "max", "mean", "stddev").show()

+-------+------------------+
|summary|      duration_sec|
+-------+------------------+
|  count|             99998|
|    min|                 0|
|    max|              8820|
|   mean| 650.5954119082381|
| stddev|469.01392166279226|
+-------+------------------+



In [8]:
# Check if there is passenger_count == 0
df_taxi.select("passenger_count").summary("count", "min", "max", "mean", "stddev").show()

+-------+------------------+
|summary|   passenger_count|
+-------+------------------+
|  count|             99998|
|    min|                 0|
|    max|                 6|
|   mean|2.1629832596651934|
| stddev|1.7398872965148953|
+-------+------------------+



In [9]:
# Does passenger_count == 0 mean idle time? Ignore for now, since not specified.

In [10]:
# Now we can remove all unnecessary columns
df_taxi = df_taxi.drop(
    col("vendor_id"),
    col("rate_code"), 
    col("store_and_fwd_flag"), 
    col("pickup_datetime"),
    col("dropoff_datetime"),
    col("passenger_count")
)

In [11]:
df_taxi.printSchema()
df_taxi.show(5)

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)
 |-- pickup_ts: long (nullable = true)
 |-- dropoff_ts: long (nullable = true)
 |-- duration_sec: long (nullable = true)

+--------------------+--------------------+----------------+---------------+-----------------+----------------+----------+----------+------------+
|           medallion|        hack_license|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude| pickup_ts|dropoff_ts|duration_sec|
+--------------------+--------------------+----------------+---------------+-----------------+----------------+----------+----------+------------+
|89D227B655E5C82AE...|BA96DE419E711691B...|      -73.978165|      40.757977|       -73.989838|       40.751171|1357053060|1357053480|         420|
|0BD7C8F

In [12]:
pip install shapely

  Using cached shapely-2.0.7-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached shapely-2.0.7-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.5 MB)
Note: you may need to restart the kernel to use updated packages.


In [13]:
import json
from shapely.geometry import shape

with open("input/nyc-boroughs.geojson", "r") as f:
    geojson_data = json.load(f)

# Extract features
features = geojson_data["features"]

# Sort features by borough code and area (descending)
def feature_sort_key(f):
    borough_code = f["properties"]["boroughCode"]
    polygon_area = shape(f["geometry"]).area
    return (borough_code, polygon_area * -1)  # negative for descending on area

features_sorted = sorted(features, key=feature_sort_key)

In [14]:
# Broadcast the GeoJSON
bc_features = spark.sparkContext.broadcast(features_sorted)

In [15]:
# Define a UDF to Map Coordinates → Borough
from shapely.geometry import Point
from pyspark.sql.functions import udf

def get_borough(lon, lat, features):
    if lon is None or lat is None:
        return None
    point = Point(float(lon), float(lat))
    for f in features:
        polygon = shape(f["geometry"])
        if polygon.contains(point):
            return f["properties"]["borough"]
    return None

def udf_get_borough(lon, lat):
    return get_borough(lon, lat, bc_features.value)

borough_udf = udf(udf_get_borough)

In [16]:
# Apply the UDF
from pyspark.sql.functions import col

df_taxi = df_taxi.withColumn("pickup_borough",
    borough_udf(col("pickup_longitude"), col("pickup_latitude"))
)

df_taxi = df_taxi.withColumn("dropoff_borough",
    borough_udf(col("dropoff_longitude"), col("dropoff_latitude"))
)

In [17]:
# Remove longitude latitude
df_taxi = df_taxi.drop(
    col("pickup_longitude"),
    col("pickup_latitude"),
    col("dropoff_longitude"),
    col("dropoff_latitude")
)

In [18]:
df_taxi.printSchema()
df_taxi.show(5)

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- pickup_ts: long (nullable = true)
 |-- dropoff_ts: long (nullable = true)
 |-- duration_sec: long (nullable = true)
 |-- pickup_borough: string (nullable = true)
 |-- dropoff_borough: string (nullable = true)

+--------------------+--------------------+----------+----------+------------+--------------+---------------+
|           medallion|        hack_license| pickup_ts|dropoff_ts|duration_sec|pickup_borough|dropoff_borough|
+--------------------+--------------------+----------+----------+------------+--------------+---------------+
|89D227B655E5C82AE...|BA96DE419E711691B...|1357053060|1357053480|         420|     Manhattan|      Manhattan|
|0BD7C8F5BA12B88E0...|9FD8F69F0804BDB55...|1357431480|1357431720|         240|     Manhattan|      Manhattan|
|0BD7C8F5BA12B88E0...|9FD8F69F0804BDB55...|1357411740|1357412040|         300|     Manhattan|      Manhattan|
|DFD2202EE08F7A8DC...|51EE87E3205C9

In [19]:
# caching the DataFrame
df_taxi.cache()
df_taxi.count()  # trigger the cache

99998

In [20]:
# partion and sort
from pyspark.sql.window import Window
import pyspark.sql.functions as F

# For cab utilization, use medallion, for taxi driver productivity, use hack_license
window_spec = Window.partitionBy("medallion").orderBy("pickup_ts")

In [21]:
# Compute Idle Time
# Compare the pickup time of the current trip to the dropoff time of the previous trip:
df_taxi = df_taxi.withColumn(
  "prev_dropoff_ts",
  F.lag("dropoff_ts").over(window_spec)
)
df_taxi = df_taxi.withColumn(
  "idle_time_sec",
  col("pickup_ts") - col("prev_dropoff_ts")
)
# Replace null with 0 for the first trip
df_taxi = df_taxi.fillna({"idle_time_sec": 0})

In [22]:
# ignore very large idle times (e.g., >4 hours) as new sessions. Maybe we do not need this.
df_taxi = df_taxi.withColumn(
  "idle_time_sec",
  F.when(col("idle_time_sec") <= 4*60*60, col("idle_time_sec")).otherwise(0)
)

In [23]:
### Required Queries:

In [24]:
# Utilization (per Taxi/Driver)
# “Fraction of time a taxi is occupied.”
#Summation of driving time vs. total time (driving + idle)

df_trip_time = df_taxi.groupBy("medallion") \
    .agg(F.sum("duration_sec").alias("sum_trip_time"))

df_idle_time = df_taxi.groupBy("medallion") \
    .agg(F.sum("idle_time_sec").alias("sum_idle_time"))

df_util = df_trip_time.join(df_idle_time, on="medallion") \
    .withColumn(
        "utilization",
        F.col("sum_trip_time") / (F.col("sum_trip_time") + F.col("sum_idle_time"))
    )

In [25]:
# Average Time to Next Fare (per Destination Borough)

# "lead" so we can shift the idle time back one row
df_taxi = df_taxi.withColumn(
  "idle_time_for_this_dropoff",
  F.lag("idle_time_sec").over(window_spec)
)
# group by dropoff_borough
df_avg_idle = df_taxi.groupBy("dropoff_borough") \
    .agg(F.avg("idle_time_for_this_dropoff").alias("avg_idle_sec"))
df_avg_idle.show()

+---------------+------------------+
|dropoff_borough|      avg_idle_sec|
+---------------+------------------+
|         Queens| 1194.029975020816|
|           NULL|1215.2891396332864|
|       Brooklyn|1170.3303509979353|
|  Staten Island|            2736.0|
|      Manhattan|1012.9720330039336|
|          Bronx| 1278.841463414634|
+---------------+------------------+



In [26]:
# Number of Trips that Start and End in the Same Borough

df_same = df_taxi.filter(col("pickup_borough") == col("dropoff_borough"))
count_same = df_same.count()

# broken down by borough
df_same.groupBy("pickup_borough").count().show()

+--------------+-----+
|pickup_borough|count|
+--------------+-----+
|        Queens| 1396|
|      Brooklyn| 1065|
| Staten Island|    1|
|     Manhattan|83560|
|         Bronx|   51|
+--------------+-----+



In [27]:
# Number of Trips from One Borough to Another

df_diff = df_taxi.filter(col("pickup_borough") != col("dropoff_borough"))
count_diff = df_diff.count()

# Per borough pair
df_diff.groupBy("pickup_borough", "dropoff_borough").count().show()

+--------------+---------------+-----+
|pickup_borough|dropoff_borough|count|
+--------------+---------------+-----+
|      Brooklyn|      Manhattan|  774|
|        Queens|          Bronx|  100|
|      Brooklyn|         Queens|  115|
|        Queens|  Staten Island|    2|
|     Manhattan|  Staten Island|    9|
|     Manhattan|       Brooklyn| 1923|
|     Manhattan|         Queens| 3943|
|     Manhattan|          Bronx|  244|
|        Queens|      Manhattan| 3698|
|         Bronx|      Manhattan|   25|
|        Queens|       Brooklyn|  597|
|         Bronx|         Queens|    2|
| Staten Island|         Queens|    1|
+--------------+---------------+-----+



In [28]:
### Save

In [29]:
df_util.coalesce(1).write.csv("output", header=True, mode="overwrite")